In [1]:
import sys
import ok
import matplotlib.pylab as plt
import matplotlib.image as mpimg
import PIL.Image as img
import numpy as np
import cv2
from IPython.display import clear_output
%matplotlib tk

In [2]:
dev = ok.okCFrontPanel()
deviceCount = dev.GetDeviceCount()
for i in range(deviceCount):
    print 'Device[{0}] Model: {1}'.format(i, dev.GetDeviceListModel(i))
    print 'Device[{0}] Serial: {1}'.format(i, dev.GetDeviceListSerial(i))

Device[0] Model: 21
Device[0] Serial: 143400091C


In [3]:
dev = ok.okCFrontPanel()
dev.OpenBySerial("")
error = dev.ConfigureFPGA("cntr_fifo.bit")
# Its a good idea to check for errors here

In [4]:
# IsFrontPanelEnabled returns true if FrontPanel is detected.
if True == dev.IsFrontPanelEnabled():
    print "FrontPanel host interface enabled."
else:
    sys.stderr.write("FrontPanel host interface not detected.")

FrontPanel host interface enabled.


In [5]:
row = 128
col = 128
count = 1
datain128 = bytearray(131072)
datain1 = bytearray(51200)

In [6]:
#im = np.array(img.open('imTest.jpg'))
#imgplot = plt.imshow(im)
im = np.zeros(shape=(row ,col))
#plt.figure(3)
#plt.imshow(im).set_cmap('gray')
#plt.show()

In [7]:
# assert reset signal to initialize the FIFO.
dev.SetWireInValue(0x10, 0xff, 0x01);
dev.UpdateWireIns();
# deactivate reset signal and activate counter.
dev.SetWireInValue(0x10, 0x00, 0x01);
dev.UpdateWireIns();   

In [8]:
# Trigger the counter
dev.ActivateTriggerIn(0x53, 0x01)
cv2.namedWindow("imFull",cv2.WINDOW_NORMAL)
cv2.namedWindow("im",cv2.WINDOW_NORMAL)
while True:
    # Check for FIFO flag
    dev.UpdateTriggerOuts()
    # If the FIFO is full, read everything and display one frame only and exit the while loop
    if dev.IsTriggered(0x6A, 0x01) == True:
        # print 'FIFO full! ', count, ' times'
        count = count + 1
        dev.ReadFromPipeOut(0xA0, datain128)
        for i in range(row):
            im[i] = datain128[i*col:i*col+col]
        im = im/255
        cv2.imshow('imFull',im)
        cv2.waitKey(1)
        dev.ActivateTriggerIn(0x53, 0x02)
        dev.SetWireInValue(0x10, 0xff, 0x01);
        dev.UpdateWireIns();
        dev.SetWireInValue(0x10, 0x00, 0x01);
        dev.UpdateWireIns(); 
        # break
    # If one frame is ready in FIFO
    if dev.IsTriggered(0x6A, 0x04) == True:
        dev.ReadFromPipeOut(0xA0, datain1)
        for i in range(row):
            im[i] = datain1[i*col:i*col+col]
        im = im/255
        cv2.waitKey(1)
        cv2.imshow('im',im)

KeyboardInterrupt: 

In [9]:
print im*255

[[ 216.  217.  218. ...,  101.  102.  103.]
 [ 104.  105.  106. ...,  229.  230.  231.]
 [ 232.  233.  234. ...,  117.  118.  119.]
 ..., 
 [ 136.  137.  138. ...,   21.   22.   23.]
 [  24.   25.   26. ...,  149.  150.  151.]
 [ 152.  153.  154. ...,   37.   38.   39.]]
